In [1]:
# !pip install 'zarr<3'
# !pip install timm
# !pip install openslide-python tiffslide
# !pip install pyarrow
# !pip install shapely
# !pip install scikit-image
# !pip install opencv-python

# # Install CuPy (will auto-detect CUDA version)
# #Install CuPy pre-built binary for CUDA 12.x
# !pip install cupy-cuda12x

In [2]:
import torch
print(f"PyTorch CUDA version: {torch.version.cuda}")

# Then install matching CuPy:
# CUDA 11.x → !pip install cupy-cuda11x
# CUDA 12.x → !pip install cupy-cuda12x

PyTorch CUDA version: 12.1


In [3]:

# Verify CuPy installation
import cupy as cp
print(f'✅ CuPy installed: {cp.cuda.runtime.getDeviceCount()} GPU(s) available')

✅ CuPy installed: 2 GPU(s) available


In [4]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## Flex 

In [5]:
import torch
from vitaminp import VitaminPFlex
from vitaminp.inference import ChannelConfig, WSIPredictor
# Setup model
device = 'cuda'
model = VitaminPFlex(model_size='large').to(device)
model.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold21_best.pth", map_location=device))
model.eval()

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ VitaminPFlex initialized with large backbone
  Architecture: Shared Encoder → 4 Separate Decoders
  Embed dim: 1024 | Decoder dims: [1024, 512, 256, 128]


VitaminPFlex(
  (backbone): DINOv2Backbone(
    (dinov2): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): M

In [6]:
# Create channel config for Xenium data
config = ChannelConfig(
    nuclear_channel=0,           # Channel 0: DAPI
    membrane_channel=[1],        # Channel 1: CD45/E-cadherin
    membrane_combination='max',  # Not really needed for single membrane channel
    channel_names={0: 'DAPI', 1: 'CD45_Ecadherin'}
)

print("\nChannel configuration:")
print(f"  Nuclear channel: 0 (DAPI)")
print(f"  Membrane channel: 1 (CD45/E-cadherin)")

# Create predictor
predictor = WSIPredictor(
    model=model,
    device=device,
    patch_size=512,
    overlap=64,
    target_mpp=0.2125,          # Xenium pixel size from your original code
    magnification=40,
    mif_channel_config=config
)

print("\nPredictor settings:")
print(f"  Patch size: 512")
print(f"  Overlap: 64")
print(f"  Target MPP: 0.2125 μm/pixel")
print(f"  Magnification: 40x")

# Run inference on combined 2-channel crop
input_path = 'test_images/combined_2channel_crop_3000x3000.ome.tif'

print(f"\n{'='*60}")
print("RUNNING INFERENCE...")
print(f"{'='*60}")
print(f"Input: {input_path}")

results = predictor.predict(
    wsi_path=input_path,
    output_dir='results_new',
    branch='mif_cell',              # Use cell branch for whole cells
    filter_tissue=False,            # Process entire crop
    clean_overlaps=True,            # Clean boundary artifacts
    save_geojson=True,              # Save GeoJSON for visualization
    detection_threshold=0.3,        # Adjust for sensitivity
    min_area_um=10.0,               # Filter small artifacts (10 μm²)
    mpp_override=0.263,            # Use Xenium pixel size
)

print(f"\n{'='*60}")
print("RESULTS")
print(f"{'='*60}")
print(f"✅ Found {results['num_detections']} cells in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")

# Check cell statistics
if results['num_detections'] > 0:
    first_cell = list(results['instances'].values())[0]
    if 'area_um' in first_cell:
        print(f"   First cell area: {first_cell['area_um']:.2f} μm² ({first_cell['area_pixels']:.0f} pixels²)")
    
    # Calculate some statistics
    import numpy as np
    areas_um = [inst['area_um'] for inst in results['instances'].values() if 'area_um' in inst]
    if areas_um:
        print(f"\n   Cell area statistics:")
        print(f"     Mean: {np.mean(areas_um):.2f} μm²")
        print(f"     Median: {np.median(areas_um):.2f} μm²")
        print(f"     Min: {np.min(areas_um):.2f} μm²")
        print(f"     Max: {np.max(areas_um):.2f} μm²")
    
    print(f"\n   Output files:")
    print(f"     • Cell instances: {output_dir}/instances.geojson")
    print(f"     • Results pickle: {output_dir}/results.pkl")

print(f"\n{'='*60}")
print("INFERENCE COMPLETE!")
print(f"{'='*60}")

2026-01-28 19:26:38 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-28 19:26:38 - WSIPredictor - INFO -   Device: cuda
2026-01-28 19:26:38 - WSIPredictor - INFO -   Model type: VitaminPFlex (single-modality)
2026-01-28 19:26:38 - WSIPredictor - INFO -   Patch size: 512
2026-01-28 19:26:38 - WSIPredictor - INFO -   Overlap: 64
2026-01-28 19:26:38 - WSIPredictor - INFO -   Magnification: 40
2026-01-28 19:26:38 - WSIPredictor - INFO -   MIF channels: Nuclear: DAPI, Membrane: max(CD45_Ecadherin)
2026-01-28 19:26:38 - WSIPredictor - INFO -    Manual MPP override: 0.2630 μm/px
2026-01-28 19:26:38 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-28 19:26:38 - WSIPredictor - INFO -    WSI MPP: 0.2630 μm/px
2026-01-28 19:26:38 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-28 19:26:38 - WSIPredictor - INFO -    Scale factor: 1.00x
2026-01-28 19:26:38 - WSIPredictor - INFO -    Min area filter: 10.0 μm² = 145 pixels²
2026-01-28 19:26:38 - WSIPredictor - INFO 


Channel configuration:
  Nuclear channel: 0 (DAPI)
  Membrane channel: 1 (CD45/E-cadherin)

Predictor settings:
  Patch size: 512
  Overlap: 64
  Target MPP: 0.2125 μm/pixel
  Magnification: 40x

RUNNING INFERENCE...
Input: test_images/combined_2channel_crop_3000x3000.ome.tif
Output: shape=(2, 3000, 3000), dtype=float32, range=[0.000, 0.029]
   Scanning 7x7 tile grid...


Processing tiles: 100%|██████████| 49/49 [00:09<00:00,  5.12it/s]
2026-01-28 19:26:48 - WSIPredictor - INFO -    ✓ Extracted 5277 instances from tiles (before cleaning)
2026-01-28 19:26:48 - WSIPredictor - INFO -    🔍 DEBUG: Tile configuration:
2026-01-28 19:26:48 - WSIPredictor - INFO -       - Tile size: 512px
2026-01-28 19:26:48 - WSIPredictor - INFO -       - Overlap: 64px
2026-01-28 19:26:48 - WSIPredictor - INFO -       - Grid: 7x7 tiles
2026-01-28 19:26:48 - WSIPredictor - INFO -    🔍 DEBUG: Cells near tile boundaries (within 64px): 2383
2026-01-28 19:26:48 - WSIPredictor - INFO -    🔍 DEBUG: Potential overlap rate: 45.2%
2026-01-28 19:26:48 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-28 19:26:48 - WSIPredictor - INFO -    🔍 DEBUG: Total cells before cleaning: 5277
2026-01-28 19:26:48 - WSIPredictor - INFO -    🔍 DEBUG: Edge cells found: 2293
2026-01-28 19:26:48 - WSIPredictor - INFO -    🔍 DEBUG: Will clean ALL instances (not just edge


RESULTS
✅ Found 3481 cells in 17.17s
   Output saved to: results_new

   Output files:


NameError: name 'output_dir' is not defined